# 第7章 文本数据

2020.06.24,   2020.06.25 review,  

In [1]:
import pandas as pd
import numpy as np

## 一、string类型的性质

### 1. string与object的区别
#### string类型和object不同之处有三：
#### ① 字符存取方法（string accessor methods，如str.count）会返回相应数据的Nullable类型，而object会随缺失值的存在而改变返回类型
#### ② 某些Series方法不能在string上使用，例如： Series.str.decode()，因为存储的是字符串而不是字节
#### ③ string类型在缺失值存储或运算时，类型会广播为pd.NA，而不是浮点型np.nan
#### 其余全部内容在当前版本下完全一致，但迎合Pandas的发展模式，我们仍然全部用string来操作字符串

### 2. string类型的转换
#### 如果将一个其他类型的容器直接转换string类型可能会出错：

In [8]:
#pd.Series([1,'1.']).astype('string') #报错 ValueError: StringArray requires a sequence of strings or pandas.NA
#pd.Series([1,2]).astype('string') #报错 ValueError: StringArray requires a sequence of strings or pandas.NA
#pd.Series([True,False]).astype('string') #报错 ValueError: StringArray requires a sequence of strings or pandas.NA

#### 当下正确的方法是分两部转换，先转为str型object，再转为string类型：

In [9]:
pd.Series([1,'1.']) # 原始类型是 int64 和 str 混合的 object
#pd.Series([1,'1.']).astype('string') #直接转换会报错 ValueError: StringArray requires a sequence of strings or pandas.NA

0     1
1    1.
dtype: object

In [10]:
pd.Series([1,'1.']).astype('str') # 第一步转为 str 的object

0     1
1    1.
dtype: object

In [11]:
pd.Series([1,'1.']).astype('str').astype('string') # 第二步转为 string 类型

0     1
1    1.
dtype: string

In [12]:
#上述转换是可逆的吗？
# 转回object肯定是没问题的, object就类似于 "不能识别的数据类型"
pd.Series([1,'1.']).astype('str').astype('string').astype('str')

0     1
1    1.
dtype: object

In [13]:
pd.Series([1,2]) #原始类型是 int64
#pd.Series([1,2]).astype('string') #直接转换会报错--ValueError: StringArray requires a sequence of strings or pandas.NA

0    1
1    2
dtype: int64

In [14]:
pd.Series([1,2]).astype('str') # 第一步转为 str 类型的 object

0    1
1    2
dtype: object

In [15]:
pd.Series([1,2]).astype('str').astype('string') # 第二步转为 string 类型

0    1
1    2
dtype: string

In [17]:
# 能否转回 int64?
pd.Series([1,2]).astype('str').astype('string').astype('str')
pd.Series([1,2]).astype('str').astype('string').astype('str').astype('int64')

0    1
1    2
dtype: int64

In [22]:
# float 类型呢?
pd.Series([1.1,2]).astype('str').astype('string').astype('str').astype('float')
# 也没问题
# 但是包含缺失值的话就需要注意了

0    1.1
1    2.0
dtype: float64

In [38]:
# 这个缺失值好像算是正确转换了的
type(pd.Series([1.1,2,np.nan]).astype('str').astype('string').astype('str').astype('float')[2]) ,type(np.nan)

(numpy.float64, float)

In [23]:
pd.Series([True,False]) # 原始类型是布尔型
#pd.Series([True,False]).astype('string') #直接转换会报错 ValueError: StringArray requires a sequence of strings or pandas.NA

0     True
1    False
dtype: bool

In [24]:
pd.Series([True,False]).astype('str') #第一步转为 str 类型的object

0     True
1    False
dtype: object

In [25]:
pd.Series([True,False]).astype('str').astype('string') #第二步转为 string 类型

0     True
1    False
dtype: string

In [32]:
# 将时间转为string
from datetime import datetime
pd.Series([datetime.now(),datetime(2008,8,8,8,8,8)])

0   2020-06-25 09:08:21.423238
1   2008-08-08 08:08:08.000000
dtype: datetime64[ns]

In [29]:
pd.Series([datetime.now(),datetime(2008,8,8,8,8,8)]).astype('str')

0    2020-06-25 09:07:39.336831
1    2008-08-08 08:08:08.000000
dtype: object

In [30]:
pd.Series([datetime.now(),datetime(2008,8,8,8,8,8)]).astype('str').astype('string') 

0    2020-06-25 09:07:51.157507
1    2008-08-08 08:08:08.000000
dtype: string

In [33]:
# 时间可以直接转为 string
pd.Series([datetime.now(),datetime(2008,8,8,8,8,8)]).astype('string') 

0    2020-06-25 09:09:01.577534
1    2008-08-08 08:08:08.000000
dtype: string

In [34]:
# 将string转为时间也可以直接转
pd.Series([datetime.now(),datetime(2008,8,8,8,8,8)]).astype('string').astype('datetime64[ns]')

0   2020-06-25 09:20:25.553656
1   2008-08-08 08:08:08.000000
dtype: datetime64[ns]

In [ ]:
# 提取日期留待第九章学习

* 但这种方法也不是特别正确, 因为会把缺失值也按字面显示转为string. 见下文中的例子

## 二、拆分与拼接

### 1. str.split方法

#### （a）分割符与str的位置元素选取

In [2]:
s = pd.Series(['a_b_c', 'c_d_e', np.nan, 'f_g_h'], dtype="string")
s

0    a_b_c
1    c_d_e
2     <NA>
3    f_g_h
dtype: string

#### 根据某一个元素分割，默认为空格

In [64]:
s.str.split('_')

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [65]:
# 对比元素级的方法--缺失值没有split的属性方法,因此会报错
s[0].split('_'),s[1].split('_'),s[3].split('_'),
#s[2].split('_'),# AttributeError: 'NAType' object has no attribute 'split'

(['a', 'b', 'c'], ['c', 'd', 'e'], ['f', 'g', 'h'])

In [230]:
# 因此, 由于缺失值的存在, Series对象的 元素级 apply 方法不能直接使用
#s.apply(lambda x: x.split('_')) 
# AttributeError: 'NAType' object has no attribute 'split'

In [66]:
# 使用apply方法的话,需要分情况
# 但是 .str.split 方法为什么能正确执行? --因为, 第一步的 .str 相当于是做了判断和转换: 如不是缺失值则转为str类型, 然后再进行下一步的split, 而缺失值则会什么都不做.
def split_func(x):
    if x is pd.NA:
        y = x
    else :
        y = x.split('_')
    return y
s.apply(lambda x: split_func(x))

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [67]:
# 考虑一个例子
s_ = pd.Series(['a<b<c', 'c>d>e', np.nan, 'f<g>h'], dtype="string")
s_.str.split('<') # 直接调用 .str.split 方法是没问题的

0    [a, b, c]
1      [c>d>e]
2         <NA>
3     [f, g>h]
dtype: object

In [68]:
# 但如果先做了 astype 类型转换, 就出现问题了-缺失值被转为了它的字面表示 <NA> ,然后被split
s_.astype('str').str.split('<')

0    [a, b, c]
1      [c>d>e]
2      [, NA>]
3     [f, g>h]
dtype: object

In [70]:
import re
#s.apply(lambda x: re.split(x,'_')) # 使用正则表达式的分割函数同样会因为缺失值导致出问题 TypeError: first argument must be string or compiled pattern

In [71]:
# 使用正则表达式的方法,也需重新定义个函数以处理缺失值--所以re的代码也不够鲁棒, 实际使用时, 应该自行增加这类判断.
def my_re_split(x,rep=''):
    if x is pd.NA:
        y = x
    else :
        y = re.split(rep,x)
    return y
s.apply(lambda x: my_re_split(x,'_'))

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [79]:
re.split('_',s[0]),re.split('_',s[0]),    re.split('_',s[3]),
# re.split('_',s[2]),#TypeError: expected string or bytes-like object

(['a', 'b', 'c'], ['a', 'b', 'c'], ['f', 'g', 'h'])

In [80]:
re.split??

Signature: re.split(pattern, string, maxsplit=0, flags=0)
Source:   
def split(pattern, string, maxsplit=0, flags=0):
    """Split the source string by the occurrences of the pattern,
    returning a list containing the resulting substrings.  If
    capturing parentheses are used in pattern, then the text of all
    groups in the pattern are also returned as part of the resulting
    list.  If maxsplit is nonzero, at most maxsplit splits occur,
    and the remainder of the string is returned as the final element
    of the list."""
    return _compile(pattern, flags).split(string, maxsplit)
File:      d:\programdata\anaconda3\lib\re.py
Type:      function


#### 这里需要注意split后的类型是object，因为现在Series中的元素已经不是string，而是list(调用split后哪怕没被分割,也是返回list)，而string类型只能含有字符串

In [75]:
# 当然,这时候仍然可以用上述的迂回方式将list转为string---注意这时缺失值 pd.NA 也被转为了字面显示的字符串,幸好该缺失值表示法有类似于转义符的表达形式 <NA>
s.apply(lambda x: my_re_split(x,'_')).astype('str').astype('string')
# 但实际上很多时候是没必要这么做的,因为我们需要的就是这个list,唯一需要的是要把list分拆成多列,或提取某个元素, 分别见下文的 expand 参数和后边的 extract 方法

0    ['a', 'b', 'c']
1    ['c', 'd', 'e']
2               <NA>
3    ['f', 'g', 'h']
dtype: string

#### str方法支持元素的选择，如果该单元格元素是列表，那么str[i]表示取出第i个元素，如果是单个元素，则先把元素转为列表再取出

In [76]:
s.str.split('_').str[1]
# 缺失值则还是返回缺失值

0       b
1       d
2    <NA>
3       g
dtype: object

In [77]:
s.str.split('_')
# 注意分拆后, 非空的单元格里的元素已经是个list了,但str[1]方法能够准确地提取到这个list的相应位置的元素--这就是Series of list对象的向量化切片方法

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [258]:
# 向量化的切片方式--1
s.str.split('_').str[1:3]

0    [b, c]
1    [d, e]
2      <NA>
3    [g, h]
dtype: object

In [259]:
# 向量化的切片方式--2
# 逆序遍历
s.str.split('_').str[::-1]
# 这么看来, 其他list的各种切片的操作都应该会被这种方法所支持

0    [c, b, a]
1    [e, d, c]
2         <NA>
3    [h, g, f]
dtype: object

In [62]:
# 字符串对象当然也是支持切片的, 但转为字符串以后就不容易取到list的元素了
s.apply(lambda x: my_re_split(x,'_')).astype('str').astype('string').str[:3]
# 注意这个奇怪的事情--pd.NA对象也被转为 string 了--这不合适--可以通过replace方法先将 <NA> 字符串替换为缺失值
# .astype('str').astype('string') 会把 pd.NA 也按字面显示转为 string, 这就不合适了吧?

0    ['a
1    <NA
2    ['d
dtype: string

In [54]:
# replace方法先将 <NA> 字符串替换为缺失值--注意这个方法是针对单元格的,而不是单元格内的内容作为字符串的 .str.replace 方法. 
type(s.apply(lambda x: my_re_split(x,'_')).astype('str').astype('string').replace('<NA>',pd.NA).str[:3][2])

pandas._libs.missing.NAType

In [273]:
s[2],type(s[2]),

(<NA>, pandas._libs.missing.NAType)

In [276]:
s.apply(lambda x: split_func(x))[2],type(s.apply(lambda x: split_func(x))[2])

(<NA>, pandas._libs.missing.NAType)

In [275]:
s.apply(lambda x: my_re_split(x,'_'))[2],type(s.apply(lambda x: my_re_split(x,'_'))[2]),

(<NA>, pandas._libs.missing.NAType)

In [284]:
s.apply(lambda x: my_re_split(x,'_')).astype('str')[2],type(s.apply(lambda x: my_re_split(x,'_')).astype('str')[2])
# 在这一步就出问题了--astype('str')会按字面显示将objec转为字符串

('<NA>', str)

In [274]:
# pd.NA变成了 string--已经出现了数据处理错误
s.apply(lambda x: my_re_split(x,'_')).astype('str').astype('string')[2],type(s.apply(lambda x: my_re_split(x,'_')).astype('str').astype('string')[2])

('<NA>', str)

In [283]:
pd.Series([1,'1.',pd.NA]).astype('str').astype('string')[2],type(pd.Series([1,'1.',pd.NA]).astype('str').astype('string')[2]) 
pd.Series([1,'1.',np.nan]).astype('str').astype('string')[2],type(pd.Series([1,'1.',np.nan]).astype('str').astype('string')[2]) 
pd.Series([1,'1.',None]).astype('str').astype('string')[2],type(pd.Series([1,'1.',None]).astype('str').astype('string')[2]) 
#pd.Series([1,2,pd.NA]).astype('str').astype('string')[2], type(pd.Series([1,2,pd.NA]).astype('str').astype('string')[2])
#pd.Series([1,2,np.nan]).astype('str').astype('string')[2], type(pd.Series([1,2,np.nan]).astype('str').astype('string')[2])
#pd.Series([True,False]).astype('string')

('None', str)

In [285]:
# astype是个比较霸道的类型转换方式, 缺乏对缺失值的有效操作方式.
s.astype??

Signature: s.astype(dtype, copy: bool = True, errors: str = 'raise') -> ~FrameOrSeries
Source:   
    def astype(
        self: FrameOrSeries, dtype, copy: bool_t = True, errors: str = "raise"
    ) -> FrameOrSeries:
        """
        Cast a pandas object to a specified dtype ``dtype``.

        Parameters
        ----------
        dtype : data type, or dict of column name -> data type
            Use a numpy.dtype or Python type to cast entire pandas object to
            the same type. Alternatively, use {col: dtype, ...}, where col is a
            column label and dtype is a numpy.dtype or Python type to cast one
            or more of the DataFrame's columns to column-specific types.
        copy : bool, default True
            Return a copy when ``copy=True`` (be very careful setting
            ``copy=False`` as changes to values then may propagate to other
            pandas objects).
        errors : {'raise', 'ignore'}, default 'raise'
            Control raising of excep

In [291]:
s.str.split('_')[:][1],  s.str.split('_')[1]# 选取效果和上述是有些区别的

(['c', 'd', 'e'], ['c', 'd', 'e'])

In [56]:
pd.Series(['a_b_c', ['a','b','c']], dtype="object")#.str[1]
#第一个元素先用list('a_b_c')转为['a','_','b','_','c'],然后再切片
#第二个元素是个list, list会正常的切片

0        a_b_c
1    [a, b, c]
dtype: object

In [295]:
pd.Series(['a_b_c', ['a','b','c']], dtype="object").str[1]

0    _
1    b
dtype: object

In [55]:
pd.Series(['a_b_c', ['a','b','c']], dtype="object").str[:2]
# str虽然看起来是转为str再操作, 但实际上, 对于是字符串的单元格,会选择该字符串相应位置的元素,而对于是list的, 则会使用切片方法

0        a_
1    [a, b]
dtype: object

#### （b）其他参数
#### expand参数控制了是否将列拆开，n参数代表最多分割多少次(分割成n+1列)

In [58]:
s.str.split??
#n : int, default -1 (all)
#    Limit number of splits in output.
#    ``None``, 0 and -1 will be interpreted as return all splits.
#expand : bool, default False
#    Expand the splitted strings into separate columns.
#
#    * If ``True``, return DataFrame/MultiIndex expanding dimensionality.
#    * If ``False``, return Series/Index, containing lists of strings.

Signature: s.str.split(pat=None, n=-1, expand=False)
Docstring:
Split strings around given separator/delimiter.

Splits the string in the Series/Index from the beginning,
at the specified delimiter string. Equivalent to :meth:`str.split`.

Parameters
----------
pat : str, optional
    String or regular expression to split on.
    If not specified, split on whitespace.
n : int, default -1 (all)
    Limit number of splits in output.
    ``None``, 0 and -1 will be interpreted as return all splits.
expand : bool, default False
    Expand the splitted strings into separate columns.

    * If ``True``, return DataFrame/MultiIndex expanding dimensionality.
    * If ``False``, return Series/Index, containing lists of strings.

Returns
-------
Series, Index, DataFrame or MultiIndex
    Type matches caller unless ``expand=True`` (see Notes).

See Also
--------
Series.str.split : Split strings around given separator/delimiter.
Series.str.rsplit : Splits string around given separator/delimiter,
  

In [57]:
s.str.split('_',expand=True)

,0,1,2
0,a,b,c
1,c,d,e
2,<NA>,<NA>,<NA>
3,f,g,h


In [301]:
s.str.split('_',n=3) #expand 默认是 False, 因此单独指定 n=3 是没有效果的

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [58]:
s.str.split('_',expand=True,n=1) # n=1 则拆分一次, expand=True 则会拆分成多列--由于只进行了一次拆分,于是被拆分成了1+1=2列

,0,1
0,a,b_c
1,c,d_e
2,<NA>,<NA>
3,f,g_h


In [59]:
s.str.split('_',expand=True,n=1)

,0,1
0,a,b_c
1,c,d_e
2,<NA>,<NA>
3,f,g_h


### 2. str.cat方法

#### （a）不同对象的拼接模式
#### cat方法对于不同对象的作用结果并不相同，其中的对象包括：单列、双列、多列

#### ① 对于单个Series而言，就是指所有的元素进行字符合并为一个字符串

In [3]:
s = pd.Series(['ab',None,'d'],dtype='string')
s

0      ab
1    <NA>
2       d
dtype: string

In [4]:
s.str.cat()

'abd'

#### 其中可选sep分隔符参数，和缺失值替代字符na_rep参数

In [5]:
s.str.cat(sep=',')
# oracle 里有 listagg(11.2及以上)和 wm_concat 两个函数实现相同功能

'ab,d'

In [6]:
s.str.cat(sep=',',na_rep='*')

'ab,*,d'

In [13]:
# 配合分组函数使用
s_= pd.DataFrame({'name':['allen',None,'mike','joy','ross','john','lisa','lily'],'gender':['male',None,'male','female','male','female','male','female']},dtype='string')
s_

,name,gender
0,allen,male
1,<NA>,<NA>
2,mike,male
3,joy,female
4,ross,male
5,john,female
6,lisa,male
7,lily,female


In [14]:
s_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    7 non-null      string
 1   gender  7 non-null      string
dtypes: string(2)
memory usage: 256.0 bytes


In [15]:
s_.groupby('gender').apply(lambda x:x['name'].str.cat(sep=',',na_rep='?'))
# 分组时会忽略缺失值--应该先用一个"安全"的字符串替换缺失值,等分组后再替换回去.

gender
female           joy,john,lily
male      allen,mike,ross,lisa
dtype: object

In [17]:
# 先用安全的字符串'unknown'填充缺失值,分组后,在cat前再把name列的'unknown'替换回缺失值
s_.fillna('unknown').groupby('gender').apply(lambda x:x['name'].replace('unknown',pd.NA).str.cat(sep=',',na_rep='?'))

gender
female            joy,john,lily
male       allen,mike,ross,lisa
unknown                       ?
dtype: object

In [ ]:
# 其实还可以进一步先排序再cat.排序时是否可以设置缺失值放在前边还是放在后边?

#### ② 对于两个Series合并而言，是对应索引的元素进行合并

In [19]:
s2 = pd.Series(['24','cc',None],dtype='string')
s2

0      24
1      cc
2    <NA>
dtype: string

In [20]:
s.str.cat(s2)
# 第二和第三个元素都成了pd.NA了--默认情况下,缺失值会传播

0    ab24
1    <NA>
2    <NA>
dtype: string

#### 同样也有相应参数，需要注意的是两个缺失值会被同时替换

In [21]:
s.str.cat(s2,sep=',',na_rep='*')

0    ab,24
1     *,cc
2      d,*
dtype: string

In [23]:
#如果只替换其中之一的缺失值,或者分别用不同的字符替换两个series的缺失值,可以先使用 replace 方法替换缺失值后, 再用 str.cat 方法拼接.
s.replace(pd.NA,'*').str.cat(s2,sep=',',na_rep='?')

0    ab,24
1     *,cc
2      d,?
dtype: string

#### ③ 多列拼接可以分为表的拼接和多Series拼接

#### 表的拼接

In [24]:
display(s)
d_ = pd.DataFrame({0:['1','3','5'],1:['5','b',None]},dtype='string')
display(d_)
s.str.cat(d_,na_rep='*')
# 将一个 Series 对象和一个等长的 DF 对象按索引进行行的拼接,会按索引对其后,

0      ab
1    <NA>
2       d
dtype: string

,0,1
0,1,5
1,3,b
2,5,<NA>


0    ab15
1     *3b
2     d5*
dtype: string

In [25]:
# 对照来看, 确实是把df的行和series的单元格都拼在一起了.
s.to_frame().join(d_,lsuffix='_l')

,0_l,0,1
0,ab,1,5
1,<NA>,3,b
2,d,5,<NA>


In [313]:
# s.to_frame().join(d_,lsuffix='_l').str.cat(na_rep='*') 
#AttributeError: 'DataFrame' object has no attribute 'str'
# 想把 DF 的行拼接到一起还不是那么直接

In [34]:
# 一个将df按行拼接起来的思路
# 先构造索引一致且值全为空字符串的Series,然后用Series拼接df的方式, 用这个Series去拼接表
df=pd.DataFrame(np.random.randint(0,20,21).reshape(3,7),index=list('abc'),columns=['C_1','C_2','C_3','C_4','C_5','C_6','C_7']).astype('str')
display(df)
s1=pd.Series(''*3,index=df.index)
display(s1)
s1.str.cat(df,sep='+',na_rep='*')

,C_1,C_2,C_3,C_4,C_5,C_6,C_7
a,5,2,18,10,2,14,2
b,15,17,6,12,2,4,10
c,1,18,12,17,8,8,9


a    
b    
c    
dtype: object

a     +5+2+18+10+2+14+2
b    +15+17+6+12+2+4+10
c     +1+18+12+17+8+8+9
dtype: object

In [35]:
# 或者, 将df的第一列单独拿出来作为 series, 用上述方法, 与 df 剩余的列进行拼接
df.C_1.str.cat(df.iloc[:,1:],sep='+',na_rep='*')

a     5+2+18+10+2+14+2
b    15+17+6+12+2+4+10
c     1+18+12+17+8+8+9
Name: C_1, dtype: object

In [36]:
# 还可以将df拆分成series, 然后用循环,进行拼接
s1=df.C_1
for col in df.columns[1:]:
    s1=s1.str.cat(df[col],sep='+')
s1

a     5+2+18+10+2+14+2
b    15+17+6+12+2+4+10
c     1+18+12+17+8+8+9
Name: C_1, dtype: object

#### 多个Series按索引拼接

In [37]:
s.str.cat([s+'0',s*2])

0    abab0abab
1         <NA>
2        dd0dd
dtype: string

In [320]:
#有没有实现该功能的类似的顶级方法? 来统一处理 Series 和 DataFrame?

#### （b）cat中的索引对齐
#### 当前版本中，如果两边合并的索引不相同且未指定join参数，默认为左连接，设置join='left'

In [38]:
s2 = pd.Series(list('abc'),index=[1,2,3],dtype='string')
s2

1    a
2    b
3    c
dtype: string

In [39]:
s.str.cat(s2,na_rep='*')

0    ab*
1     *a
2     db
dtype: string

In [42]:
# 如果二者索引没有对齐,相当于左连接,然后再拼接--由于左连接可能会引入缺失值,如果索引完全不对齐则会把s的每个元素与一个na_rep参数指定的字符串拼接起来.
s.str.cat(s1,sep='_',na_rep='*')

0    ab_*
1     *_*
2     d_*
dtype: string

In [43]:
# 不指定na_rep参数,则会导致缺失值的扩散
s.str.cat(s1)

0    <NA>
1    <NA>
2    <NA>
dtype: string

In [44]:
s1,s

(a     5+2+18+10+2+14+2
 b    15+17+6+12+2+4+10
 c     1+18+12+17+8+8+9
 Name: C_1, dtype: object,
 0      ab
 1    <NA>
 2       d
 dtype: string)

In [45]:
# 更换s1和s2的位置, 发现确实是左连接
s1.str.cat(s,na_rep='*')

a     5+2+18+10+2+14+2*
b    15+17+6+12+2+4+10*
c     1+18+12+17+8+8+9*
Name: C_1, dtype: object

## 三、替换
#### 广义上的替换，就是指str.replace函数的应用，fillna是针对缺失值的替换，上一章已经提及
#### 提到替换，就不可避免地接触到正则表达式，这里默认读者已掌握常见正则表达式知识点，若对其还不了解的，可以通过[这份资料](https://regexone.com/)来熟悉

### 1. str.replace的常见用法

In [46]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'],dtype="string")
s

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

#### 第一个值写r开头的正则表达式，后一个写替换的字符串

In [47]:
s.str.replace(r'^[AB]','***') #把字符串开头(大写)的A或B替换为"***"

0       ***
1       ***
2         C
3    ***aba
4    ***aca
5          
6      <NA>
7      CABA
8       dog
9       cat
dtype: string

In [51]:
s.str.replace(r'[(A)(B)]','***') # 分组替换, 把字符串当中(大写)的A或B都替换为"***"

0           ***
1           ***
2             C
3        ***aba
4        ***aca
5              
6          <NA>
7    C*********
8           dog
9           cat
dtype: string

### 2. 子组与函数替换

#### 通过正整数调用子组（0返回字符本身，从1开始才是子组）

In [52]:
s.str.replace(r'([ABC])(\w+)',repl=lambda x:x.group(2)[1:]+'*')
#repl : str or callable
#    Replacement string or a callable. The callable is passed the regex
#    match object and must return a replacement string to be used.
#    See :func:`re.sub`.

0       A
1       B
2       C
3     ba*
4     ca*
5        
6    <NA>
7     BA*
8     dog
9     cat
dtype: string

In [53]:
s

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

In [332]:
s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0))#[0:]+'*')
# group(0) 返回字符串本身--没有匹配到的也是返回整个字符串, 如果是缺失值则还是返回缺失值

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

In [333]:
s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1))#[0:]+'*')
# group(1) 返回匹配到的第一个子组--没有匹配的则继续返回整个字符串

0       A
1       B
2       C
3       A
4       B
5        
6    <NA>
7       C
8     dog
9     cat
dtype: string

In [334]:
s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2))#[0:]+'*')
# group(2) 返回匹配到的第二个子组--没有匹配的则继续返回整个字符串
# 特别地,只匹配到一个子组的, 返回的是第一个子组呢,还是整个字符串呢?

0       A
1       B
2       C
3     aba
4     aca
5        
6    <NA>
7     ABA
8     dog
9     cat
dtype: string

In [91]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'],dtype="string")
dfs=s.to_frame()
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0))"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0))
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1))"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1))
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2))"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2))
#dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(3))"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(3)) # IndexError: no such group
#dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(4))"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(4))
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:])"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:])
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0)[1:]+'****')"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0)[1:]+'****')#012689列都没有拼接****,但是有返回值
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1)[1:]+'****')"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1)[1:]+'****')
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:]+'****')"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:]+'****')
dfs.T

,0,1,2,3,4,5,6,7,8,9
0,A,B,C,Aaba,Baca,,<NA>,CABA,dog,cat
"s.str.replace(r'([ABC])+(\w+)',lambda x:x.group(0))",A,B,C,aba,aca,,<NA>,A,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0))",A,B,C,Aaba,Baca,,<NA>,CABA,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1))",A,B,C,A,B,,<NA>,C,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2))",A,B,C,aba,aca,,<NA>,ABA,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:])",A,B,C,ba,ca,,<NA>,BA,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0)[1:]+'****')",A,B,C,aba****,aca****,,<NA>,ABA****,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1)[1:]+'****')",A,B,C,****,****,,<NA>,****,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:]+'****')",A,B,C,ba****,ca****,,<NA>,BA****,dog,cat


In [59]:
who

col	 d_	 df	 dfs	 np	 pd	 s	 s1	 s2	 
s_	 


In [79]:
# 还需进一步研究...............
# 把ss的第一个元素设置为
ss = pd.Series(['Aa', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'],dtype="string")
ss.str.replace('([ABC])([abc])',lambda x:x.group(0)+'='+x.group(1)+'+'+x.group(2))#[0:]+'*')
#ss.str.replace('([ABC])',lambda x:x.group(0)+','+x.group(1))#[0:]+'*')


0      Aa=A+a
1           B
2           C
3    Aa=A+aba
4    Ba=B+aca
5            
6        <NA>
7        CABA
8         dog
9         cat
dtype: string

In [75]:
ss.str.replace('([ABC])',lambda x:print(x))#[0:]+'*')

<re.Match object; span=(0, 1), match='A'>
<re.Match object; span=(0, 1), match='B'>
<re.Match object; span=(0, 1), match='C'>
<re.Match object; span=(0, 1), match='A'>
<re.Match object; span=(0, 1), match='B'>
<re.Match object; span=(0, 1), match='C'>
<re.Match object; span=(1, 2), match='A'>
<re.Match object; span=(2, 3), match='B'>
<re.Match object; span=(3, 4), match='A'>


0       a
1        
2        
3     aba
4     aca
5        
6    <NA>
7        
8     dog
9     cat
dtype: string

In [86]:
ss = pd.Series(['0Aa', '1B', '2C', '3Aaba', '4Baca','5', np.nan, '7CABA', '8dog', '9cat'],dtype="string")
ss.str.replace('([ABC])([abc])',lambda x:x.group(0)+'='+x.group(1)+'+'+x.group(2))#[0:]+'*')


0      0Aa=A+a
1           1B
2           2C
3    3Aa=A+aba
4    4Ba=B+aca
5            5
6         <NA>
7        7CABA
8         8dog
9         9cat
dtype: string

In [107]:
s.str.replace??

Signature: s.str.replace(pat, repl, n=-1, case=None, flags=0, regex=True)
Docstring:
Replace occurrences of pattern/regex in the Series/Index with
some other string. Equivalent to :meth:`str.replace` or
:func:`re.sub`.

Parameters
----------
pat : str or compiled regex
    String can be a character sequence or regular expression.
repl : str or callable
    Replacement string or a callable. The callable is passed the regex
    match object and must return a replacement string to be used.
    See :func:`re.sub`.
n : int, default -1 (all)
    Number of replacements to make from start.
case : bool, default None
    Determines if replace is case sensitive:

    - If True, case sensitive (the default if `pat` is a string)
    - Set to False for case insensitive
    - Cannot be set if `pat` is a compiled regex.

flags : int, default 0 (no flags)
    Regex module flags, e.g. re.IGNORECASE. Cannot be set if `pat` is a compiled
    regex.
regex : bool, default True
    Determines if assumes the 

In [105]:
re.sub??

Signature: re.sub(pattern, repl, string, count=0, flags=0)
Source:   
def sub(pattern, repl, string, count=0, flags=0):
    """Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used."""
    return _compile(pattern, flags).sub(repl, string, count)
File:      c:\programdata\anaconda3\lib\re.py
Type:      function


#### 利用?P<....>表达式可以对子组命名调用

In [377]:
s.str.replace(r'(?P<one>[ABC])(?P<two>\w+)',lambda x:x.group('two')[1:]+'*')

0       A
1       B
2       C
3     ba*
4     ca*
5        
6    <NA>
7     BA*
8     dog
9     cat
dtype: string

In [378]:
s.str.replace(r'(?P<one>[ABC])(?P<two>\w+)',lambda x:print(x)) # lambda 中的 x 表示前边的正则表达式匹配到的子串

<re.Match object; span=(0, 4), match='Aaba'>

<re.Match object; span=(0, 4), match='Baca'>

<re.Match object; span=(0, 4), match='CABA'>


0       A
1       B
2       C
3        
4        
5        
6    <NA>
7        
8     dog
9     cat
dtype: string

In [379]:
s

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

In [380]:
ss=s+'FF123'
ss

0       AFF123
1       BFF123
2       CFF123
3    AabaFF123
4    BacaFF123
5        FF123
6         <NA>
7    CABAFF123
8     dogFF123
9     catFF123
dtype: string

In [381]:
ss.str.replace(r'(?P<one>[ABC])(?P<two>\w+)',lambda x:print(x.group(0),x.group(1),x.group(2))) # lambda 中的 x 表示前边的正则表达式匹配到的子串

AFF123 A FF123

BFF123 B FF123

CFF123 C FF123

AabaFF123 A abaFF123

BacaFF123 B acaFF123

CABAFF123 C ABAFF123


0            
1            
2            
3            
4            
5       FF123
6        <NA>
7            
8    dogFF123
9    catFF123
dtype: string

### 3. 关于str.replace的注意事项
#### 首先，要明确str.replace和replace并不是一个东西：
#### str.replace针对的是object类型或string类型，默认是以正则表达式为操作，目前暂时不支持DataFrame上使用
#### replace针对的是任意类型的序列或数据框，如果要以正则表达式替换，需要设置regex=True，该方法通过字典可支持多列替换
#### 但现在由于string类型的初步引入，用法上出现了一些问题，这些issue有望在以后的版本中修复


#### （a）str.replace赋值参数不得为pd.NA
#### 这听上去非常不合理，例如对满足某些正则条件的字符串替换为缺失值，直接更改为缺失值在当下版本就会报错

In [164]:
#pd.Series(['A','B'],dtype='string').str.replace(r'[A]',pd.NA) #报错 TypeError: repl must be a string or callable
#pd.Series(['A','B'],dtype='O').str.replace(r'[A]',pd.NA) #报错 TypeError: repl must be a string or callable

#### 此时，可以先转为object类型再转换回来，曲线救国：

In [92]:
pd.Series(['A','B'],dtype='string').astype('O').replace(r'[A]',pd.NA,regex=True).astype('string')

0    <NA>
1       B
dtype: string

#### 至于为什么不用replace函数的regex替换（但string类型replace的非正则替换是可以的），原因在下面一条

#### （b）对于string类型Series，在使用replace函数时不能使用正则表达式替换
#### 该bug现在还未修复

In [93]:
pd.Series(['A','B'],dtype='string').replace(r'[A]','C',regex=True)
# dtype='string' 没有被替换

0    A
1    B
dtype: string

In [94]:
pd.Series(['A','B'],dtype='O').replace(r'[A]','C',regex=True)
# dtype='o' 时可以替换

0    C
1    B
dtype: object

In [95]:
pd.Series(['A','B'],dtype='str').replace(r'[A]','C',regex=True)
# dtype='str' 也可以被替换

0    C
1    B
dtype: object

In [104]:
# 对于 int 类型
pd.Series([1,2,3],dtype='int').replace(r'[1]','C',regex=True)
# 直接替换并不成功

0    1
1    2
2    3
dtype: int32

In [107]:
pd.Series([1,2,3],dtype='int').astype('str').replace(r'[1]','C',regex=True)
# 转为 str 后可以成功替换.

0    C
1    2
2    3
dtype: object

In [108]:
pd.Series([1,2,3],dtype='int').astype('O').replace(r'[1]','C',regex=True)
# 转为 object 后, 并未成功替换

0    1
1    2
2    3
dtype: int64

In [110]:
pd.Series([12,23,34],dtype='int').astype('str').replace(r'[1]','C',regex=True)
# 

0    C2
1    23
2    34
dtype: object

In [111]:
pd.Series([12,23,34],dtype='int').astype('str').replace(r'1','C',regex=True)
# 

0    C2
1    23
2    34
dtype: object

In [116]:
pd.Series([1213,23,34],dtype='int').astype('str').replace(r'1','9',regex=True).astype('int')
# 可以将某个数字替换为其他数字, 但需要用 astype 再转回数字.

0    9293
1      23
2      34
dtype: int32

#### （c）string类型序列如果存在缺失值，不能使用replace替换

In [385]:
pd.Series(['A',np.nan],dtype='string').replace('A','B') #报错--没有报错啊

0       B
1    <NA>
dtype: string

In [389]:
pd.Series(['A',np.nan],dtype='str').replace('A','B') # str 类型也没有报错
pd.Series(['A',np.nan],dtype='O').replace('A','B') # O 类型也没有报错

0      B
1    NaN
dtype: object

In [386]:
pd.Series(['A',np.nan],dtype='string').str.replace('A','B')

0       B
1    <NA>
dtype: string

#### 综上，概况的说，除非需要赋值元素为缺失值（转为object再转回来），否则请使用str.replace方法

## 四、子串匹配与提取

### 1. str.extract方法
从单元格里提取符合传入的正则表达式的子串, 也可以用来将一列扩张为多列.

#### （a）常见用法

In [390]:
display(pd.Series(['10-87', '10-88', '10-89'],dtype="string"))
pd.Series(['10-87', '10-88', '10-89'],dtype="string").str.extract(r'([\d]{2})-([\d]{2})')

0    10-87
1    10-88
2    10-89
dtype: string

,0,1
0,10,87
1,10,88
2,10,89


In [176]:
s.str.extract??

Signature: s.str.extract(pat, flags=0, expand=True)
Docstring:
Extract capture groups in the regex `pat` as columns in a DataFrame.

For each subject string in the Series, extract groups from the
first match of regular expression `pat`.

Parameters
----------
pat : str
    Regular expression pattern with capturing groups.
flags : int, default 0 (no flags)
    Flags from the ``re`` module, e.g. ``re.IGNORECASE``, that
    modify regular expression matching for things like case,
    spaces, etc. For more details, see :mod:`re`.
expand : bool, default True
    If True, return DataFrame with one column per capture group.
    If False, return a Series/Index if there is one capture group
    or DataFrame if there are multiple capture groups.

Returns
-------
DataFrame or Series or Index
    A DataFrame with one row for each subject string, and one
    column for each group. Any capture group names in regular
    expression pat will be used for column names; otherwise
    capture group number

#### 通过在正则表达式中使用子组名来为拆分后的列增加列名

In [391]:
pd.Series(['10-87', '10-88', '-89'],dtype="string").str.extract(r'(?P<start>[\d]{2})-(?P<end>[\d]{2})')

,start,end
0,10,87
1,10,88
2,<NA>,<NA>


#### 利用?正则标记选择部分提取

In [392]:
pd.Series(['10-87', '10-88', '10-',  '-89'],dtype="string").str.extract(r'(?P<name_1>[\d]{2})?-(?P<name_2>[\d]{2})?')
# 第一组后的?表示该组是可能缺失的,第一组没匹配到的也能够正确提取第二组.
# 在第二组后加上?则第二组匹配不到的也会正确提取第一组.

,name_1,name_2
0,10,87
1,10,88
2,10,<NA>
3,<NA>,89


In [393]:
pd.Series(['10-87', '10-88', '10-'],dtype="string").str.extract(r'(?P<name_1>[\d]{2})-(?P<name_2>[\d]{2})?')

,name_1,name_2
0,10,87
1,10,88
2,10,<NA>


#### （b）expand参数（默认为True）

#### 对于一个子组的Series，如果expand设置为False，则返回Series，若大于一个子组，则expand参数无效，全部返回DataFrame
#### 对于一个子组的Index，如果expand设置为False，则返回提取后的Index，若大于一个子组且expand为False，报错

In [394]:
s = pd.Series(["a1", "b2", "c3"], ["A11", "B22", "C33"], dtype="string")
s#.index

A11    a1
B22    b2
C33    c3
dtype: string

In [395]:
s.str.extract(r'([\w])')

,0
A11,a
B22,b
C33,c


In [396]:
s.str.extract(r'([\w])',expand=False)

A11    a
B22    b
C33    c
dtype: string

In [397]:
s.str.extract(r'([\w])',expand=True)

,0
A11,a
B22,b
C33,c


In [398]:
('m'+s).str.extract(r'([\w])([\w])([\w])',expand=False) # 大于1个组, 因此 expand=False 参数就没法起作用了--这和split方法中需要先指定expand, n才能起作用是不一致的.

,0,1,2
A11,m,a,1
B22,m,b,2
C33,m,c,3


In [399]:
s.index.str.extract(r'([\w])')

,0
0,A
1,B
2,C


In [400]:
s.index.str.extract(r'([\w])',expand=False)

Index(['A', 'B', 'C'], dtype='object')

In [401]:
# 提取两个子组类型,都提取导了,因此自然会分成两列
s.index.str.extract(r'([\w])([\d])')

,0,1
0,A,1
1,B,2
2,C,3


In [409]:
# s.index.str.extract(r'([\w])([\d])([\w])') # 第三个子组会继续沿用第二个
# s.index.str.extract(r'([\w])([\d])([\d])') # 同上
s.index.str.extract(r'([\w])([\d])([\w])([\w])') #再增加一个就都变成 NaN 了

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN


In [413]:
s.index.str.extract(r'([\w])([\d])([\d])')#([\d])') #三个没问题, 再增加一个就都变成 NaN 了

,0,1,2
0,A,1,1
1,B,2,2
2,C,3,3


In [415]:
# 索引也支持提取
#s.index.str.extract(r'([\w])([\d])',expand=False) #报错 ValueError: only one regex group is supported with Index
s.index.str.extract(r'([\w])([\d])',expand=True)

,0,1
0,A,1
1,B,2
2,C,3


In [416]:
# 对于索引, 也可以先使用 to_series 函数(注意s小写)转为Series再提取--这在从复合型单层索引构造层次索引时会用到.
s.index.to_series().str.extract(r'([\w])([\d])',expand=False)

,0,1
A11,A,1
B22,B,2
C33,C,3


### 2. str.extractall方法

#### 与extract只匹配第一个符合条件的表达式不同，extractall会找出所有符合条件的字符串，并建立多级索引（即使只找到一个）

In [282]:
s = pd.Series(["a1a2", "b1", "c1"], index=["A", "B", "C"],dtype="string")
display(s)
two_groups = '(?P<letter>[a-z])(?P<digit>[0-9])'
s.str.extract(two_groups, expand=True)

A    a1a2
B      b1
C      c1
dtype: string

,letter,digit
A,a,1
B,b,1
C,c,1


In [283]:
s.str.extractall(two_groups)
# 将匹配到的多个符合条件的子串分为了多行 -- 这个处理方式非常好, 因为如果分为多列的话会导致大量的单元格是缺失值

letter digit
  match             
A 0          a     1
  1          a     2
B 0          b     1
C 0          c     1

In [284]:
s['A']='a1'
s.str.extractall(two_groups)

,,letter,digit
,match,,
A,0,a,1
B,0,b,1
C,0,c,1


In [285]:
s.str.extractall??

Signature: s.str.extractall(pat, flags=0)
Docstring:
For each subject string in the Series, extract groups from all
matches of regular expression pat. When each subject string in the
Series has exactly one match, extractall(pat).xs(0, level='match')
is the same as extract(pat).

Parameters
----------
pat : str
    Regular expression pattern with capturing groups.
flags : int, default 0 (no flags)
    A ``re`` module flag, for example ``re.IGNORECASE``. These allow
    to modify regular expression matching for things like case, spaces,
    etc. Multiple flags can be combined with the bitwise OR operator,
    for example ``re.IGNORECASE | re.MULTILINE``.

Returns
-------
DataFrame
    A ``DataFrame`` with one row for each match, and one column for each
    group. Its rows have a ``MultiIndex`` with first levels that come from
    the subject ``Series``. The last level is named 'match' and indexes the
    matches in each item of the ``Series``. Any capture group names in
    regular expre

#### 如果想查看第i层匹配，可使用xs方法

In [286]:
s = pd.Series(["a1a2", "b1b2", "c1c2"], index=["A", "B", "C"],dtype="string")
s.str.extractall(two_groups).xs(1,level='match')

,letter,digit
A,a,2
B,b,2
C,c,2


### 3. str.contains和str.match
#### 前者的作用为检测是否包含某种正则模式

In [287]:
pd.Series(['1', None, '3a', '3b', '03c'], dtype="string").str.contains(r'[0-9][a-z]')

0    False
1     <NA>
2     True
3     True
4     True
dtype: boolean

In [291]:
pd.Series(['1', None, '3a3', '3b', '03c'], dtype="string").str.contains(r'^[0-9][a-z]$')

0    False
1     <NA>
2    False
3     True
4    False
dtype: boolean

#### 可选参数为na

In [292]:
pd.Series(['1', None, '3a', '3b', '03c'], dtype="string").str.contains('a', na=True)#False)
# na=False参数会把缺失值的返回值设置为False,反之则会设置为True -- 可以根据业务需求设置

0    False
1     True
2     True
3    False
4    False
dtype: boolean

#### str.match与其区别在于，match依赖于python的re.match，检测内容为是否从头开始包含该正则模式

In [293]:
pd.Series(['1', None, '3a_', '3b', '03c'], dtype="string").str.match(r'[0-9][a-z]',na=False)

0    False
1    False
2     True
3     True
4    False
dtype: boolean

In [294]:
pd.Series(['1', None, '_3a', '3b', '03c'], dtype="string").str.match(r'[0-9][a-z]',na=False)

0    False
1    False
2    False
3     True
4    False
dtype: boolean

In [298]:
astr="""begin--
1this is a string,
2with 2 lines,
3or 3, maybe.
"""
pd.Series([astr,'1'+astr, None, '3a_', '3b', '03c'], dtype="string").str.match(r'[0-9][a-z]',na=False)
pd.Series([astr,'1'+astr, None, '3a_', '3b', '03c'], dtype="string").str.match(r'[0-9][a-z]',na=False,flags=re.M)

0    False
1     True
2    False
3     True
4     True
5    False
dtype: boolean

In [303]:
# 补充说明: 字符串的 find 方法并不支持正则表达式
pd.Series(['1', None, '3a_', '3b', '03c'], dtype="string").str.find(r'[0-9][a-z]')

0      -1
1    <NA>
2      -1
3       0
4      -1
dtype: Int64

In [307]:
pd.Series(['1', None, '3a_', '3b', '03c'], dtype="string").str.find('3a')

0      -1
1    <NA>
2       0
3      -1
4      -1
dtype: Int64

## 五、常用字符串方法

### 1. 过滤型方法
#### （a）str.strip
#### 常用于过滤空格

In [308]:
pd.Series(list('abc'),index=[' space1  ','space2  ','  space3'],dtype="string").index.str.strip()

Index(['space1', 'space2', 'space3'], dtype='object')

#### （b）str.lower和str.upper

In [309]:
pd.Series('A',dtype="string").str.lower()

0    a
dtype: string

In [310]:
pd.Series('a',dtype="string").str.upper()

0    A
dtype: string

In [318]:
print(dir(pd.Series('hey joy, How\'re you doing?',dtype="string").str))

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__frozen', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_doc_args', '_freeze', '_get_series_list', '_inferred_dtype', '_is_categorical', '_is_string', '_make_accessor', '_orig', '_parent', '_validate', '_wrap_result', 'capitalize', 'casefold', 'cat', 'center', 'contains', 'count', 'decode', 'encode', 'endswith', 'extract', 'extractall', 'find', 'findall', 'get', 'get_dummies', 'index', 'isalnum', 'isalpha', 'isdecimal', 'isdigit', 'islower', 'isnumeric', 'isspace', 'istitle', 'isupper', 'join', 'len', 'ljust', 'lower', 'lstrip', 'match', 'normalize', 'pad', 'partition', 'repeat', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'r

In [333]:
pd.Series('hey joy, How\'re you doing?',dtype="string").str.zfill??

Signature: str.zfill(self, width, /)
Docstring:
Pad a numeric string with zeros on the left, to fill a field of the given width.

The string is never truncated.
Type:      method_descriptor


#### （c）str.swapcase, str.title和str.capitalize
#### 分别表示交换字母大小写,每个单词首字母大写和整个字符串的首字母大写

In [334]:
pd.Series('abCD',dtype="string").str.swapcase()

0    ABcd
dtype: string

In [335]:
pd.Series('abCD',dtype="string").str.capitalize()

0    Abcd
dtype: string

In [336]:
# 将每个单词的首字母大写,其他字母小写
pd.Series('hey joy, How are you doing?',dtype="string").str.title()

0    Hey Joy, How Are You Doing?
dtype: string

In [337]:
# 只将第一个单词的首字母大写,其他的字母小写
pd.Series('hey joy, How\'re you doing?',dtype="string").str.capitalize()

0    Hey joy, how're you doing?
dtype: string

### 2. isnumeric方法
#### 检查每一位是否都是数字，请问如何判断是否是数值？（问题二）

In [338]:
pd.Series(['1.2','1','-0.3','a',np.nan],dtype="string").str.isnumeric()
# 对于负数和浮点数给出了错误的答案

0    False
1     True
2    False
3    False
4     <NA>
dtype: boolean

In [340]:
pd.Series(['1.2','1','-0.3','a',np.nan],dtype="string").str.isdecimal()
# 同样错误

0    False
1     True
2    False
3    False
4     <NA>
dtype: boolean

In [341]:
pd.Series(['1.2','1','-0.3','a',np.nan],dtype="string").str.isdigit()
# 错误

0    False
1     True
2    False
3    False
4     <NA>
dtype: boolean

In [343]:
pd.Series(['1.2','1','-0.3','a',np.nan],dtype="string").str.isalnum??

Signature: str.isalnum(self, /)
Docstring:
Return True if the string is an alpha-numeric string, False otherwise.

A string is alpha-numeric if all characters in the string are alpha-numeric and
there is at least one character in the string.
Type:      method_descriptor


In [364]:
# 先判断是否是缺失值, 如果不是, 再使用正则表达式匹配浮点数模式: 以零或一个负号(-)开头,接下来至少包含一个数字,最后以零或一个"小数点加零或多个数字"结尾
pd.Series(['42','1.223','1.','-0.3456','a1','1a',np.nan,None,pd.NA],dtype="string").apply(lambda x:True if x is not pd.NA and re.match('^-?\d+(\.\d*)?$',x) else False)

0     True
1     True
2     True
3     True
4    False
5    False
6    False
7    False
8    False
dtype: bool

## 六、问题与练习
### 1. 问题

#### 【问题一】 str对象方法和df/Series对象方法有什么区别？

针对的对象不同. 
* str 对象方法是将单元格内的内容当作字符串,来执行针对字符串的各种操作.  
* 而 pandas 对象 DataFrame 和 Series 的对象方法,操作的多是 DF 的列的所有元素(如 sum, min 等聚合函数),或者是单元格, 并且是对单元格内的内容整体进行操作.
    * 例如, str.replace可以把单元格内的字符串的一部分替换掉,但是df.replace是把整个单元格内的内容替换掉(并且要数据类型一致)

In [370]:
who

astr	 col	 d_	 day2c	 df	 df1	 df2	 dfs	 l1	 
month2c	 np	 num2c	 num_to_c	 num_to_c1	 num_to_char	 pd	 re	 s	 
s1	 s2	 s_	 s_string	 ss	 two_groups	 


In [371]:
df

,C_1,C_2,C_3,C_4,C_5,C_6,C_7
a,5,2,18,10,2,14,2
b,15,17,6,12,2,4,10
c,1,18,12,17,8,8,9


In [374]:
# 上述df中的单元格内的对象是str类型.
df.applymap(lambda x: x+f'的平方是{int(x)**2}')

,C_1,C_2,C_3,C_4,C_5,C_6,C_7
a,5的平方是25,2的平方是4,18的平方是324,10的平方是100,2的平方是4,14的平方是196,2的平方是4
b,15的平方是225,17的平方是289,6的平方是36,12的平方是144,2的平方是4,4的平方是16,10的平方是100
c,1的平方是1,18的平方是324,12的平方是144,17的平方是289,8的平方是64,8的平方是64,9的平方是81


In [395]:
#　再举一个例子
from datetime import datetime
st=pd.Series([datetime.now()]) # datetime类型
st.values[0]
sts = pd.concat([st,st.astype('str'),st.astype('string')])
display(sts) # 字面上看来, 三个元素完全一致,但实际上三个元素的数据类型不同 

0    2020-06-26 19:09:09.338082
0    2020-06-26 19:09:09.338082
0    2020-06-26 19:09:09.338082
dtype: object

In [396]:
sts.replace('2020-06-26 19:09:09.338082','--') # 于是使用Series的replace,就会只替换字符串,而略过时间类型的第一个元素

0    2020-06-26 19:09:09.338082
0                            --
0                            --
dtype: object

In [418]:
# 但是如果是用时间去替换..
from dateutil.parser import parse
sts.replace(parse('2020-06-26 19:09:09.338082'),'--') 
# 就只会替换第一个

0                            --
0    2020-06-26 19:09:09.338082
0    2020-06-26 19:09:09.338082
dtype: object

In [419]:
sts.str.replace('2020-06-26 19:09:09.338082','--') # 但是使用 str.replace, 奇怪的事情发生了.... 第一个元素被替换为了空

0    NaN
0     --
0     --
dtype: object

In [421]:
sts.str.replace('--','--')
# 是因为str方法的原因--这算是个bug吗?

0                           NaN
0    2020-06-26 19:09:09.338082
0    2020-06-26 19:09:09.338082
dtype: object

In [422]:
sts.astype('str').str.replace('--','--')

0    2020-06-26 19:09:09.338082
0    2020-06-26 19:09:09.338082
0    2020-06-26 19:09:09.338082
dtype: object

In [398]:
# 这种情况下, 先转一下数据类型, 再操作就能得到期望的结果了.
sts.astype('str').str.replace('2020-06-26 19:09:09.338082','--') # 但是使用 str.replace, 奇怪的事情发生了....

0    --
0    --
0    --
dtype: object

#### 【问题二】 给出一列string类型，如何判断单元格是否是数值型数据？

In [427]:
s_string = pd.Series(['1','0','-2.123','0.2','-0.3','1a','a1','a',np.nan],dtype="string")
display(s_string)# .str.extract(r'\d')  #ValueError: pattern contains no capture groups

0         1
1         0
2    -2.123
3       0.2
4      -0.3
5        1a
6        a1
7         a
8      <NA>
dtype: string

In [428]:
s_string.str.isdecimal()

0     True
1     True
2    False
3    False
4    False
5    False
6    False
7    False
8     <NA>
dtype: boolean

In [432]:
# 用正则表达式匹配
s_string.str.contains(r'^-?(\d+)(\.\d+)?$')

0     True
1     True
2     True
3     True
4     True
5    False
6    False
7    False
8     <NA>
dtype: boolean

#### 【问题三】 rsplit方法的作用是什么？它在什么场合下适用？

In [443]:
s.str.rsplit??

Signature: s.str.rsplit(pat=None, n=-1, expand=False)
Docstring:
Split strings around given separator/delimiter.

Splits the string in the Series/Index from the end,
at the specified delimiter string. Equivalent to :meth:`str.rsplit`.

Parameters
----------
pat : str, optional
    String or regular expression to split on.
    If not specified, split on whitespace.
n : int, default -1 (all)
    Limit number of splits in output.
    ``None``, 0 and -1 will be interpreted as return all splits.
expand : bool, default False
    Expand the splitted strings into separate columns.

    * If ``True``, return DataFrame/MultiIndex expanding dimensionality.
    * If ``False``, return Series/Index, containing lists of strings.

Returns
-------
Series, Index, DataFrame or MultiIndex
    Type matches caller unless ``expand=True`` (see Notes).

See Also
--------
Series.str.split : Split strings around given separator/delimiter.
Series.str.rsplit : Splits string around given separator/delimiter,
    st

#### 【问题四】 在本章的第二到第四节分别介绍了字符串类型的5类操作，请思考它们各自应用于什么场景？

* 拆分与拼接
    * str.split--拆分
    * str.cat--拼接
* 替换
* 子串匹配与提取
    * extract--从pandas对象的一列中提取信息成为新的列,例如将手机号码,邮件从文本中提取出来.
    * exractall--提取所有能够匹配到的子串,并建立多级索引
    * contain--判断包含某个特定格式的子串
    * match--字符串以某个特定子串开头
* 过滤与判断

In [450]:
# 将文末参考文献按字段整理成表格形式
#　国标引文.txt 文件来自cnki的导出题录, 导出为国标GB/T 7714-2015 参考文献格式.
!type data\国标引文.txt
# 中文字符, 需要指定encoding

[1]姊佸啗.鍩轰簬鏁板瓧浜烘枃鐨勬櫤鎱у浘涔﹂�嗗缓璁捐矾寰勬帰绱�[J].鎵嶆櫤,2020(18):241.

[2]寮犲┓.鍙版咕鍦板尯鏁板瓧浜烘枃鐮旂┒鐗硅壊涓庡疄璺靛惎绀篬J].楂樻牎鍥句功棣嗗伐浣�,2020,40(04):41-45.

[3]鏉ㄤ匠棰�,閭撶拹鑺�,璁搁懌.瑙呮睙鍗椾匠棣旓細澶氭簮鍙や粖鏂囨湰鏁版嵁铻嶅悎鐨勬勃涓婇ギ椋熷浘璋辨瀯寤篬J/OL].鍥句功棣嗚�哄潧:1-9[2020-06-26].http://kns.cnki.net/kcms/detail/44.1306.G2.20200616.1533.002.html.

[4]濮氬暩鍗�,璐烘櫒鑺�,寰愬瓭濞�,鍏ㄧ煶宄�.闈㈠悜鏁板瓧浜烘枃鐨勫浘涔﹂�嗕紬鍖呭钩鍙版瀯寤虹爺绌垛�斺�斾互涓婃捣鍥句功棣嗗巻鍙叉枃鐚�浼楀寘骞冲彴涓轰緥[J].鍥句功棣嗘潅蹇�,2020,39(06):105-112.

[5]鑼冩�傜孩,璧电函娲�.鍩轰簬鐭ヨ瘑鍥捐氨鐨勫彜绫嶆暟瀛楀寲鐮旂┒鍓嶆部鐑�鐐瑰強婕斿寲瓒嬪娍鍒嗘瀽[J].鍑虹増骞胯��,2020(11):85-87.

[6]寰愭櫒椋�,鍙舵捣褰�,鍖呭钩.鍩轰簬娣卞害瀛︿範鐨勬柟蹇楃墿浜ц祫鏂欏疄浣撹嚜鍔ㄨ瘑鍒�妯″瀷鏋勫缓鐮旂┒[J/OL].鏁版嵁鍒嗘瀽涓庣煡璇嗗彂鐜�:1-17[2020-06-26].http://kns.cnki.net/kcms/detail/10.1478.G2.20200609.1040.006.html.

[7]鍒樻槺褰�,鍚存枌,鐧藉┓.鍙よ瘲璇嶅浘璋辩殑鏋勫缓鍙婂垎鏋愮爺绌禰J].璁＄畻鏈虹爺绌朵笌鍙戝睍,2020,57(06):1252-1268.

[8]宸寸壒,閭撳悰.鏁板瓧浜烘枃瑙嗗煙涓嬫垜鍥介珮鏍″彛杩版牎鍙叉。妗堝缓璁捐矾寰勬�濊�僛J].鍏板彴涓栫晫,2020(06):31-34.

[9]鏉庢澗娑�,寮犲崼涓�,宸﹀��.鏁板瓧浜烘枃瑙嗚�掍笅浜烘枃鐮旂┒鑰呭埄鐢ㄦ。妗堥�嗚棌鐨勮�屼负涓庢縺鍔辩爺绌禰J].灞辫タ妗ｆ��,2020(03):77-97.

[10]濮滆偛褰�,鏉庨泤鑼�.鍩轰簬鏁板瓧浜烘枃瑙嗚�掔殑鈥滄儏鎰熲�斺�旀椂绌衡�濇ā鍨嬫帰鏋怺J].鍐滀笟鍥句功鎯呮姤瀛︽姤,2020,32(06):23-33.

[11]姊佺户鏂�,姹熷窛,鐜嬩笢娉�.鍩轰簬澶氱壒寰佽

In [451]:
# 第一步, 指定编码,读取文本
ref=open('data\国标引文.txt',encoding='utf8').read()
print(ref[:1000])

[1]梁军.基于数字人文的智慧图书馆建设路径探索[J].才智,2020(18):241.

[2]张婷.台湾地区数字人文研究特色与实践启示[J].高校图书馆工作,2020,40(04):41-45.

[3]杨佳颖,邓璐芗,许鑫.觅江南佳馔：多源古今文本数据融合的沪上饮食图谱构建[J/OL].图书馆论坛:1-9[2020-06-26].http://kns.cnki.net/kcms/detail/44.1306.G2.20200616.1533.002.html.

[4]姚啸华,贺晨芝,徐孝娟,全石峰.面向数字人文的图书馆众包平台构建研究——以上海图书馆历史文献众包平台为例[J].图书馆杂志,2020,39(06):105-112.

[5]范桂红,赵纯洋.基于知识图谱的古籍数字化研究前沿热点及演化趋势分析[J].出版广角,2020(11):85-87.

[6]徐晨飞,叶海影,包平.基于深度学习的方志物产资料实体自动识别模型构建研究[J/OL].数据分析与知识发现:1-17[2020-06-26].http://kns.cnki.net/kcms/detail/10.1478.G2.20200609.1040.006.html.

[7]刘昱彤,吴斌,白婷.古诗词图谱的构建及分析研究[J].计算机研究与发展,2020,57(06):1252-1268.

[8]巴特,邓君.数字人文视域下我国高校口述校史档案建设路径思考[J].兰台世界,2020(06):31-34.

[9]李松涛,张卫东,左娜.数字人文视角下人文研究者利用档案馆藏的行为与激励研究[J].山西档案,2020(03):77-97.

[10]姜育彦,李雅茹.基于数字人文视角的“情感——时空”模型探析[J].农业图书情报学报,2020,32(06):23-33.

[11]梁继文,江川,王东波.基于多特征融合的先秦典籍汉英句子对齐研究[J/OL].数据分析与知识发现:1-13[2020-06-26].http://kns.cnki.net/kcms/detail/10.1478.G2.20200603.0948.002.html.

[12]卢丹丹,聂云霞.数字人文视角下地方特色档案资源开发路径[J/OL].山西档案:1-6[2020-06-26].http://kns.cnki.net/kcms

In [454]:
# 第二步,按换行符分割后转为Series对象,以便于使用Series的 .str 类方法
refs=pd.Series(ref.split('\n')).replace('',pd.NA).dropna()
refs.head()
# 有空行,可以先 drop

0         [1]梁军.基于数字人文的智慧图书馆建设路径探索[J].才智,2020(18):241.
2    [2]张婷.台湾地区数字人文研究特色与实践启示[J].高校图书馆工作,2020,40(04)...
4    [3]杨佳颖,邓璐芗,许鑫.觅江南佳馔：多源古今文本数据融合的沪上饮食图谱构建[J/OL]....
6    [4]姚啸华,贺晨芝,徐孝娟,全石峰.面向数字人文的图书馆众包平台构建研究——以上海图书馆历...
8    [5]范桂红,赵纯洋.基于知识图谱的古籍数字化研究前沿热点及演化趋势分析[J].出版广角,2...
dtype: object

In [455]:
# 第三步,拆分
##第一次拆分,使用 .
refs=refs.str.split(r'.',expand=True)
##第二次拆分,使用 , 将未分拆的列进一步拆分,并与其他列join
refs=refs[[0,1]].join(refs[2].str.split(',',expand=True),rsuffix='_')
##第三次拆分,选取有用的非空列,将未拆分的列进一步拆分,并于其他有用的列join
refs=refs[['0','1','0_','1_']].join(refs[2].str.split(':',expand=True),rsuffix='_')
# 第四步,丢掉空行
refs=refs[['0','1','0_','1_',1]].dropna()
refs.head()

,0,1,0_,1_,1
2,[2]张婷,台湾地区数字人文研究特色与实践启示[J],高校图书馆工作,2020,41-45
6,"[4]姚啸华,贺晨芝,徐孝娟,全石峰",面向数字人文的图书馆众包平台构建研究——以上海图书馆历史文献众包平台为例[J],图书馆杂志,2020,105-112
12,"[7]刘昱彤,吴斌,白婷",古诗词图谱的构建及分析研究[J],计算机研究与发展,2020,1252-1268
18,"[10]姜育彦,李雅茹",基于数字人文视角的“情感——时空”模型探析[J],农业图书情报学报,2020,23-33
24,"[13]Tanja Wissik,Jennifer Edmond,Frank Fischer...",国际视野下的数字人文教育——基础设施视角的社区驱动型数字人文课程登记中心[J],图书馆论坛,2020,1-27


In [456]:
# 第五步,为拆分后的列添加列名
refs.columns=['作者','标题','期刊','年','页码']
refs

,作者,标题,期刊,年,页码
2,[2]张婷,台湾地区数字人文研究特色与实践启示[J],高校图书馆工作,2020,41-45
6,"[4]姚啸华,贺晨芝,徐孝娟,全石峰",面向数字人文的图书馆众包平台构建研究——以上海图书馆历史文献众包平台为例[J],图书馆杂志,2020,105-112
12,"[7]刘昱彤,吴斌,白婷",古诗词图谱的构建及分析研究[J],计算机研究与发展,2020,1252-1268
18,"[10]姜育彦,李雅茹",基于数字人文视角的“情感——时空”模型探析[J],农业图书情报学报,2020,23-33
24,"[13]Tanja Wissik,Jennifer Edmond,Frank Fischer...",国际视野下的数字人文教育——基础设施视角的社区驱动型数字人文课程登记中心[J],图书馆论坛,2020,1-27
...,...,...,...,...,...
974,"[488]郭利敏,葛亮,刘悦如",卷积神经网络在古籍汉字识别中的应用实践[J],图书馆论坛,2019,142-148
982,"[492]陆燕,卢章平",美国高校图书馆的服务转型与发展——CLSEP2018访学报告[J],图书情报研究,2019,68-73
984,"[493]郑丽央,许春漫",美国高校图书馆数字学术馆员队伍建设及启示[J],情报资料工作,2019,100-112
986,"[494]王辉,罗茜,陈圆,谭光辉",数字时代人文学科的命运——“开明论坛”第四期讨论会综述[J],绵阳师范学院学报,2019,109-114+144


In [458]:
# 再使用 srt.replace 方法将各列里无关的信息用正则表达式替换掉--作者
refs.作者=refs.作者.str.replace(r'\[\d+\]','')
refs.标题=refs.标题.str.replace(r'\[J\]','')
refs

,作者,标题,期刊,年,页码
2,张婷,台湾地区数字人文研究特色与实践启示,高校图书馆工作,2020,41-45
6,"姚啸华,贺晨芝,徐孝娟,全石峰",面向数字人文的图书馆众包平台构建研究——以上海图书馆历史文献众包平台为例,图书馆杂志,2020,105-112
12,"刘昱彤,吴斌,白婷",古诗词图谱的构建及分析研究,计算机研究与发展,2020,1252-1268
18,"姜育彦,李雅茹",基于数字人文视角的“情感——时空”模型探析,农业图书情报学报,2020,23-33
24,"Tanja Wissik,Jennifer Edmond,Frank Fischer,Fra...",国际视野下的数字人文教育——基础设施视角的社区驱动型数字人文课程登记中心,图书馆论坛,2020,1-27
...,...,...,...,...,...
974,"郭利敏,葛亮,刘悦如",卷积神经网络在古籍汉字识别中的应用实践,图书馆论坛,2019,142-148
982,"陆燕,卢章平",美国高校图书馆的服务转型与发展——CLSEP2018访学报告,图书情报研究,2019,68-73
984,"郑丽央,许春漫",美国高校图书馆数字学术馆员队伍建设及启示,情报资料工作,2019,100-112
986,"王辉,罗茜,陈圆,谭光辉",数字时代人文学科的命运——“开明论坛”第四期讨论会综述,绵阳师范学院学报,2019,109-114+144


In [459]:
# 再把refs拼接成参考文献样式
refs=refs.reset_index()
refs.head()

,index,作者,标题,期刊,年,页码
0,2,张婷,台湾地区数字人文研究特色与实践启示,高校图书馆工作,2020,41-45
1,6,"姚啸华,贺晨芝,徐孝娟,全石峰",面向数字人文的图书馆众包平台构建研究——以上海图书馆历史文献众包平台为例,图书馆杂志,2020,105-112
2,12,"刘昱彤,吴斌,白婷",古诗词图谱的构建及分析研究,计算机研究与发展,2020,1252-1268
3,18,"姜育彦,李雅茹",基于数字人文视角的“情感——时空”模型探析,农业图书情报学报,2020,23-33
4,24,"Tanja Wissik,Jennifer Edmond,Frank Fischer,Fra...",国际视野下的数字人文教育——基础设施视角的社区驱动型数字人文课程登记中心,图书馆论坛,2020,1-27


In [462]:
# [1]肖鹏,彭嗣禹,王蕾. 基本原则与关键问题——学术型图书馆馆员如何启动数字人文项目[J]. 图书馆论坛, 2017, 卷缺失(期缺失): 20-25.
ref_= pd.Series('[',index=refs.index).str.cat(refs['index'].astype('str')).str.cat(refs.作者, sep='] ').str.cat(refs.标题,sep='. ').str.cat(refs.期刊,sep='[J]. ').str.cat(refs.年,sep=', ').str.cat(refs.页码,sep=': ').to_list()
ref_[:20]

['[2] 张婷. 台湾地区数字人文研究特色与实践启示[J]. 高校图书馆工作, 2020: 41-45',
 '[6] 姚啸华,贺晨芝,徐孝娟,全石峰. 面向数字人文的图书馆众包平台构建研究——以上海图书馆历史文献众包平台为例[J]. 图书馆杂志, 2020: 105-112',
 '[12] 刘昱彤,吴斌,白婷. 古诗词图谱的构建及分析研究[J]. 计算机研究与发展, 2020: 1252-1268',
 '[18] 姜育彦,李雅茹. 基于数字人文视角的“情感——时空”模型探析[J]. 农业图书情报学报, 2020: 23-33',
 '[24] Tanja Wissik,Jennifer Edmond,Frank Fischer,Franciska de Jong,Stefania Scagliola,Andrea Scharnhorst,Hendrik Schmeer,Walter Scholger,Leon Wessels,郑炜楠,肖鹏. 国际视野下的数字人文教育——基础设施视角的社区驱动型数字人文课程登记中心[J]. 图书馆论坛, 2020: 1-27',
 '[32] 朱思苑,卢章平. 美国高校图书馆数字人文中心网站建设对我国的启示[J]. 图书情报研究, 2020: 47-54',
 '[38] 邓君,孙绍丹,王阮,宋先智. 美国29所高校数字人文项目研究内容解析[J]. 情报资料工作, 2020: 31-40',
 '[40] 马昭仪,何捷,刘帅帅. 中国古典叙事文学的时空叙事数字模型研究——以《李娃传》为例[J]. 地球信息科学学报, 2020: 967-977',
 '[42] 许刚,秦昆. “空间综合人文学与社会科学的昨天、今天和明天”沙龙纪要[J]. 地球信息科学学报, 2020: 1176-1177',
 '[58] 王丽华,刘炜,刘圣婴. 数字人文的理论化趋势前瞻[J]. 中国图书馆学报, 2020: 17-23',
 '[60] 夏翠娟. 面向人文研究的“数据基础设施”建设——试论图书馆学对数字人文的方法论贡献[J]. 中国图书馆学报, 2020: 24-37',
 '[62] 李惠,侯君明,陈涛,朱庆华,刘炜. 星汉窈渺——书信网络中蕴藏的人际关系挖掘[J]. 图书馆杂志, 2020: 86-92+80',
 '[64] 施晓华,王

In [ ]:
# 一些问题: 编号应该从1开始;卷期缺失需加进去;结尾应该再加一个'.';

### 2. 练习
#### 【练习一】 现有一份关于字符串的数据集，请解决以下问题：
#### （a）现对字符串编码存储人员信息（在编号后添加ID列），使用如下格式：“×××（名字）：×国人，性别×，生于×年×月×日”
#### （b）将（a）中的人员生日信息部分修改为用中文表示（如一九七四年十月二十三日），其余返回格式不变。
#### （c）将（b）中的ID列结果拆分为原列表相应的5列，并使用equals检验是否一致。

In [59]:
pd.read_csv('data/String_data_one.csv',index_col='人员编号').head()

,姓名,国籍,性别,出生年,出生月,出生日
人员编号,,,,,,
1,aesfd,2,男,1942,8,10
2,fasefa,5,女,1985,10,4
3,aeagd,4,女,1946,10,15
4,aef,4,男,1999,5,13
5,eaf,1,女,2010,6,24


In [463]:
df1=pd.read_csv('data/String_data_one.csv',index_col='人员编号')#.reset_index()
df1.head(20)

,姓名,国籍,性别,出生年,出生月,出生日
人员编号,,,,,,
1,aesfd,2,男,1942,8,10
2,fasefa,5,女,1985,10,4
3,aeagd,4,女,1946,10,15
4,aef,4,男,1999,5,13
5,eaf,1,女,2010,6,24
6,faefdf,1,男,1942,1,21
7,daf,4,女,1957,11,5
8,faefdf,3,男,1998,4,8
9,fs,3,女,1957,4,14


In [464]:
df1['ID']=df1.姓名.str.cat(pd.Series('(名字)',index=df1.index)).str.cat(df1.国籍.astype('str'),sep=':').str.cat(df1.性别,sep='国人,性别').str.cat(df1.出生年.astype('str'),sep=',生于').str.cat(df1.出生月.astype('str'),sep='年').str.cat(df1.出生日.astype('str'),sep='月').str.cat(pd.Series('日',index=df1.index))

In [465]:
df1['ID'].head()

人员编号
1     aesfd(名字):2国人,性别男,生于1942年8月10日
2    fasefa(名字):5国人,性别女,生于1985年10月4日
3    aeagd(名字):4国人,性别女,生于1946年10月15日
4       aef(名字):4国人,性别男,生于1999年5月13日
5       eaf(名字):1国人,性别女,生于2010年6月24日
Name: ID, dtype: object

In [616]:
df1.出生年.apply(lambda x:list(str(x))).str[0]#.replace('','一')

人员编号
1       1
2       1
3       1
4       1
5       2
       ..
1996    1
1997    1
1998    2
1999    2
2000    1
Name: 出生年, Length: 2000, dtype: object

In [466]:
num2c={'1':'一','2':'二','3':'三','4':'四','5':'五','6':'六','7':'七','8':'八','9':'九','0':'零'}

In [467]:
df1.出生年

人员编号
1       1942
2       1985
3       1946
4       1999
5       2010
        ... 
1996    1984
1997    1943
1998    2018
1999    2005
2000    1962
Name: 出生年, Length: 2000, dtype: int64

In [468]:
df1.出生年=df1.出生年.astype('str').str.replace(r'(\d)(\d)(\d)(\d)',lambda x:x.group(1)).replace(num2c).str.cat(df1.出生年.astype('str').str.replace(r'(\d)(\d)(\d)(\d)',lambda x:x.group(2)).replace(num2c)).str.cat(df1.出生年.astype('str').str.replace(r'(\d)(\d)(\d)(\d)',lambda x:x.group(3)).replace(num2c)).str.cat(df1.出生年.astype('str').str.replace(r'(\d)(\d)(\d)(\d)',lambda x:x.group(4)).replace(num2c))
df1.出生年

人员编号
1       一九四二
2       一九八五
3       一九四六
4       一九九九
5       二零一零
        ... 
1996    一九八四
1997    一九四三
1998    二零一八
1999    二零零五
2000    一九六二
Name: 出生年, Length: 2000, dtype: object

In [471]:
# 改进,使用"+"来匹配一个或多个数字
# 年份必然有四个数字
%time df1.出生年.astype('str').str.replace(r'(\d)(\d)(\d)(\d)',lambda x:x.group(4)).replace(num2c)

Wall time: 3 ms


人员编号
1       一九四二
2       一九八五
3       一九四六
4       一九九九
5       二零一零
        ... 
1996    一九八四
1997    一九四三
1998    二零一八
1999    二零零五
2000    一九六二
Name: 出生年, Length: 2000, dtype: object

In [470]:
l1=['1', '9', '4', '2']
pd.Series(l1).replace(num2c).str.cat()

'一九四二'

In [472]:
# 改进为适用于不定长的年份,但耗时久
%time df1.出生年.astype('str').apply(lambda x:list(x)).apply(lambda x:pd.Series(x).replace(num2c).str.cat())

Wall time: 1.65 s


人员编号
1       一九四二
2       一九八五
3       一九四六
4       一九九九
5       二零一零
        ... 
1996    一九八四
1997    一九四三
1998    二零一八
1999    二零零五
2000    一九六二
Name: 出生年, Length: 2000, dtype: object

In [473]:
# 继续改进--耗时较长
df1.出生年.astype('str').apply(lambda x:pd.Series(list(x)).replace(num2c).str.cat())

人员编号
1       一九四二
2       一九八五
3       一九四六
4       一九九九
5       二零一零
        ... 
1996    一九八四
1997    一九四三
1998    二零一八
1999    二零零五
2000    一九六二
Name: 出生年, Length: 2000, dtype: object

In [474]:
# 出生月
month2c={'1':'一','2':'二','3':'三','4':'四','5':'五','6':'六','7':'七','8':'八','9':'九','10':'十','11':'十一','12':'十二'}
df1.出生月.astype('str').apply(lambda x:pd.Series(x).replace(month2c).str.cat()).head(20)

人员编号
1      八
2      十
3      十
4      五
5      六
6      一
7     十一
8      四
9      四
10     九
11     四
12     六
13     七
14     十
15     十
16     二
17     四
18     八
19     十
20     六
Name: 出生月, dtype: object

In [475]:
# 出生日
# 解决方案类似出生月,将31以下的数字和对应的汉字组成字典.
day2c={'1':'一','2':'二','3':'三','4':'四','5':'五','6':'六','7':'七','8':'八','9':'九',
       '10':'十','11':'十一','12':'十二','13':'十三','14':'十四','15':'十五','16':'十六','17':'十七','18':'十八','19':'十九',
       '20':'二十','21':'二十一','22':'二十二','23':'二十三','24':'二十四','25':'二十五','26':'二十六','27':'二十七','28':'二十八','29':'二十九',
       '30':'二十','31':'三十一'}
df1.出生日.astype('str').apply(lambda x:pd.Series(x).replace(day2c).str.cat()).head(20)

人员编号
1       十
2       四
3      十五
4      十三
5     二十四
6     二十一
7       五
8       八
9      十四
10     十五
11     十二
12     十八
13      三
14      三
15     二十
16     十三
17    二十八
18     十二
19      二
20      三
Name: 出生日, dtype: object

In [ ]:
# 一个问题:
#         如何将阿拉伯数字转为中文表示?

In [476]:
# 先定义一个函数: 将阿拉伯数字逐位转为中文
def num_to_c(num):
    listnum=list(str(num))
    num_dict={"0":u"零","1":u"一","2":u"二","3":u"三","4":u"四","5":u"五","6":u"六","7":u"七","8":u"八","9":u"九"}
    lst=[]
    for i in listnum:
        lst.append(num_dict[i])
    c_num=''.join(lst)
    return c_num
num_to_c(90064212544)

'九零零六四二一二五四四'

In [477]:
# 在上述函数基础上,逆序分别增加十百千万亿等单位
def num_to_c1(num):
    listnum=list(str(num))
    listnum.reverse()
    num_dict={"0":u"零","1":u"一","2":u"二","3":u"三","4":u"四","5":u"五","6":u"六","7":u"七","8":u"八","9":u"九"}
    p=['','十','百','千','万','十','百','千','亿','十','百','千','兆','十','百','千']
    lst=[]
    for i in range(len(listnum)):
        lst.append(num_dict[listnum[len(listnum)-i-1]])
        lst.append(p[len(listnum)-i-1])
    c_num=''.join(lst)
    return c_num
num_to_c1(542154521544)
# 存在的问题:
##       1.for 循环需要改进得更加易读 
##       2.如果中间存在0, 例如 1024 ,应该转为 一千零二十四, 1204 应该转为一千二百零四,1002 应该转为一千零一.
##       3.

'五千四百二十一亿五千四百五十二万一千五百四十四'

In [478]:
num_to_c1(540050544)
#　这是错误的

'五亿四千零百零十五万零千五百四十四'

In [479]:
# 
df11=df1['ID'].str.split(r'\(名字\)\:',expand=True)
df11=df11.iloc[:,0].to_frame().join(df11[1].str.split('国人,性别',expand=True),rsuffix='_')
df11=df11.iloc[:,:2].join(df11[1].str.split(',生于',expand=True),rsuffix='_')
df11=df11.iloc[:,:3].join(df11[1].str.split('年',expand=True),rsuffix='_')
df11=df11.iloc[:,:4].join(df11[1].str.split('月',expand=True),rsuffix='_')
df11=df11.iloc[:,:5].join(df11[1].str.split('日',expand=True),rsuffix='_')
df11=df11.iloc[:,:6]

df11.columns=df1.columns[:6]
df11

,姓名,国籍,性别,出生年,出生月,出生日
人员编号,,,,,,
1,aesfd,2,男,1942,8,10
2,fasefa,5,女,1985,10,4
3,aeagd,4,女,1946,10,15
4,aef,4,男,1999,5,13
5,eaf,1,女,2010,6,24
...,...,...,...,...,...,...
1996,sdf,5,男,1984,4,17
1997,hx,1,男,1943,7,16
1998,drg,5,女,2018,4,6


In [480]:
df11.equals(pd.read_csv('data/String_data_one.csv',index_col='人员编号').astype('str'))
# 确实是和刚读入的数据是相等的

True

#### 【练习二】 现有一份半虚拟的数据集，第一列包含了新型冠状病毒的一些新闻标题，请解决以下问题：
#### （a）选出所有关于北京市和上海市新闻标题的所在行。

In [60]:
pd.read_csv('data/String_data_two.csv').head()

,col1,col2,col3
0,鄂尔多斯市第2例确诊患者治愈出院,19,363.6923
1,云南新增2例，累计124例,-67,-152.281
2,武汉协和医院14名感染医护出院,-86,325.6221
3,山东新增9例，累计307例,-74,-204.9313
4,上海开学日期延至3月,-95,4.05


In [481]:
df2=pd.read_csv('data/String_data_two.csv')

In [482]:
# 北京
df2[df2.col1.str.contains('北京')]
#上海
df2[df2.col1.str.contains('上海')]
# 北京或上海
df2[df2.col1.str.contains('[上海]|[北京]')]

,col1,col2,col3
4,上海开学日期延至3月,-95,4.05
5,北京新增25例确诊病例，累计确诊253例,-4,-289.1719
6,上海新增10例，累计243例,2,-73.7105
8,好消息！青海3名新冠肺炎患者出院,50,-87.4806
19,海南新增11例累计89例,3,-135.2709
...,...,...,...
475,青海新增0例确诊病例，疑似新增1例,-40,-264.0617
479,海南新增3例，累计46例,-9,-240.5715
480,青海新增0例确诊病例,-32,-343.4422
497,河北新增17例，累计65例,-54,69.6604


#### （b）求col2的均值。

In [484]:
# 方法１：　直接定义一个函数来处理
lst=[]
for v in df2.col2.values:
    try:
        lst.append(int(v))
    except:
        lst.append(0)
lst
np.array(lst).mean()

-1.016

In [485]:
# 方法2:　利用问题 2 的方法识别出数字, 然后把能转为数字的转为数字,其他转为0后,求均值
#df2[['col2','col3  ']].apply(lambda x: float(x) if x.str.contains(r'^-?(\d+)(\.\d+)?$') else 0).mean()
df2['col2'][df2['col2'].str.contains(r'^-?(\d+)(\.\d+)?$')].astype('float').mean()
# 不一样,这是由于这种方法把不能识别为数字的排除在外了, 求的是能识别为数字的行的均值.

-1.0221327967806841

In [486]:
# 找出有问题的行
df2['col2'][~df2['col2'].str.contains(r'^-?(\d+)(\.\d+)?$')]

309    0-
396    9`
485    /7
Name: col2, dtype: object

In [487]:
df2['col2'][df2['col2'].str.contains(r'^-?(\d+)(\.\d+)?$')].astype('float').sum()/len(df2)
# 用总和除以行数才一致.

-1.016

In [488]:
df2['col2'][df2['col2'].str.contains(r'^-?(\d+)(\.\d+)?$')].astype('float').sum()/(len(df2)-3)
# 如果除以的是总行数减去有问题的行数,结果就是直接使用mean得到的.

-1.0221327967806841

#### （c）求col3的均值。

In [489]:
lst=[]
for v in df2['col3  '].values:
    try:
        lst.append(float(v))
    except:
        lst.append(0)
lst
np.array(lst).mean()

-1.1849898000000014

In [490]:
df2.columns
# 第三列的列名有空格.

Index(['col1', 'col2', 'col3  '], dtype='object')

In [491]:
df2['col3  '][df2['col3  '].str.contains(r'^-?(\d+)(\.\d+)?$')].astype('float').sum()/(len(df2)-len(df2['col3  '][~df2['col3  '].str.contains(r'^-?(\d+)(\.\d+)?$')]))


-1.1921426559356132

In [492]:
df2['col3  '][df2['col3  '].str.contains(r'^-?(\d+)(\.\d+)?$')].astype('float').sum()/(len(df2))


-1.1849897999999996